In [1]:
from qdrant_client import QdrantClient
import openai
import os
from dotenv import load_dotenv

load_dotenv()

client = QdrantClient(url="http://localhost:6333")

openai_client = openai.Client(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [2]:
dimensions = 1536
embedding_model = "text-embedding-3-small"
collection_name = "gates_notes"

In [7]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=dimensions, distance=Distance.COSINE),
)

True

In [ ]:
import os
import PyPDF2
from qdrant_client.models import PointStruct

# Read a PDF file and split its content into chunks of text
def read_pdf_chunks(file_path, chunk_size=1000):
    chunks = []
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            # Split text into chunks
            for i in range(0, len(text), chunk_size):
                chunks.append(text[i:i+chunk_size])
    return chunks

# List all files in the directory
pdf_directory = './gates_notes'
pdf_files = [file for file in os.listdir(pdf_directory) if file.endswith('.pdf')]

# Process each PDF file
for pdf_file in pdf_files:
    year = pdf_file[:4]

    pdf_file_path = os.path.join(pdf_directory, pdf_file)
    chunk_size = 1000  # Number of characters per chunk

    pdf_chunks = read_pdf_chunks(pdf_file_path, chunk_size)

    result = openai_client.embeddings.create(input=pdf_chunks, model=embedding_model, dimensions=dimensions)

    # Format data using PointStruct
    points = [
        PointStruct(
            id=idx,
            vector=data.embedding,
            payload={
                "year": year,
                "title": pdf_file,
                "text": ''.join(pdf_chunks),
            },
        )
        for idx, (data) in enumerate(result.data)
    ]

    print(points)

    client.upsert(collection_name, points)


In [29]:
def query_qdrant(query, top_k=1):

    embedded_query = openai_client.embeddings.create(
        input=query,
        model=embedding_model,
        dimensions=dimensions,
    ).data[0].embedding
    
    query_results = client.search(
        collection_name=collection_name,
        query_vector=(
            embedded_query
        ),
        limit=top_k,
    )
    
    return query_results

In [37]:
query_results = query_qdrant('Highest impact year')
for i, article in enumerate(query_results):
    print(article)

id=22 version=12 score=0.3554935 payload={'text': "Downloads  Home Ideas / 2023 Gates Foundation Annual Letter /\nThis page is available in: \x00 \x00 \x00 \x00 Deutsch English Español Français \x00ह\x00दी⽇本 中⽂\nDoes our foundation have too\nmuch inﬂuence? Here’s how I see\nit.\nMark Suzman sees a medical app demonstration at the Veerangana Awanti Bai Women\nHospital in India. ©Gates Archive/Mansi Midha\nShareAbout us Our work Ideas Search5/6/24, 8:49 PM 2023 Gates Foundation Annual Letter: Our Inﬂuence | Bill & Melinda Gates Foundation\nhttps://www.gatesfoundation.org/ideas/articles/2023-gates-foundation-annual-letter 1/13Jan 16, 2023\n21min read\nBy \nCEO, Board Member, Bill & Melinda Gates\nFoundationMark Suzman\n2023 Gates Foundation Annual Letter\nDuring the COVID-19 pandemic, the remarkable, unprecedented progress made over\nthe previous two decades in global health and development has stalled, and in many\ncases even reversed. Sadly, instead of accelerating efforts to fight infe

In [38]:
texts = [r.payload['text'] for r in query_results]
input_text = ' '.join(texts)

In [40]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

custom_prompt = "What did the Gates Foundation accomplish that year?"

template = PromptTemplate(template="{query} Context: {context}", input_variables=["query", "context"])
prompt_with_context = template.invoke({"query": custom_prompt, "context": input_text})

llm = ChatOpenAI(temperature=0.7)
results = llm.invoke(prompt_with_context)
print(results.content)

In 2023, the Gates Foundation approved a budget of US$8.3 billion, a 15% increase over the previous year's payout. They are on track to meet their commitment of increasing their annual payout to US$9 billion by 2026. The foundation also created a new, expanded board of trustees to hold them accountable and ensure diverse perspectives are incorporated into their decisions. They continue to focus on their core mission of helping to ensure that every person has the chance to lead a healthy, productive life. The foundation is doubling down on their commitment despite the challenges posed by the COVID-19 pandemic and the stagnation of progress in global health and development.
